## Text Summarization Using Langchain 

In [1]:
import os 
open_api_key="sk-prNQ08"
os.environ["OPENAI_API_KEY"]=open_api_key

## StuffDocumentChain Text Summarization

In [2]:
from PyPDF2 import PdfReader

In [3]:
# provide the path of  pdf file/files.
pdfreader = PdfReader('/Users/mariyajose/Documents/Github_projects/text_summarisation/The Weight of Days.pdf')

In [4]:
from typing_extensions import Concatenate
# read text from pdf
text = ''
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        text += content

In [5]:
text

'The Weight of DaysJohn sat by the window, watching the world pass him by. His apartment, small and sparsely furnished, had become a place of isolation, a cocoon that sheltered him from the judgmental eyes of the outside world. The ticking of the clock on the wall echoed through the silence, a relentless reminder of time slipping away, of days blending into weeks, and weeks into months. Unemployment had a way of stretching time, making it feel both fleeting and interminable.It wasn’t always like this. There was a time when John woke up with purpose, when his alarm clock was the harbinger of a new day filled with responsibilities, challenges, and small victories. He worked as a graphic designer at a mid-sized advertising agency, where his creativity was both appreciated and monetized. He enjoyed the rhythm of his work, the deadlines that pushed him to be his best, and the camaraderie of his colleagues. They were like a second family, sharing not just work but also jokes, frustrations, a

In [6]:
from langchain.docstore.document import Document

In [7]:
docs = [Document(page_content=text)]
docs

[Document(page_content='The Weight of DaysJohn sat by the window, watching the world pass him by. His apartment, small and sparsely furnished, had become a place of isolation, a cocoon that sheltered him from the judgmental eyes of the outside world. The ticking of the clock on the wall echoed through the silence, a relentless reminder of time slipping away, of days blending into weeks, and weeks into months. Unemployment had a way of stretching time, making it feel both fleeting and interminable.It wasn’t always like this. There was a time when John woke up with purpose, when his alarm clock was the harbinger of a new day filled with responsibilities, challenges, and small victories. He worked as a graphic designer at a mid-sized advertising agency, where his creativity was both appreciated and monetized. He enjoyed the rhythm of his work, the deadlines that pushed him to be his best, and the camaraderie of his colleagues. They were like a second family, sharing not just work but also

In [8]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(temperature=0, model_name='gpt-4')

/var/folders/50/37d92zzj16z4b8zf67ydlch40000gn/T/ipykernel_31033/1958061401.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm = ChatOpenAI(temperature=0, model_name='gpt-4')


In [9]:
from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.docstore.document import Document

In [10]:
template = '''Write a concise and short summary of the following story.
story: `{text}`
'''
prompt = PromptTemplate(
    input_variables=['text'],
    template=template
)

In [11]:
chain = load_summarize_chain(
    llm,
    chain_type='stuff',
    prompt=prompt,
    verbose=False
)
output_summary = chain.run(docs)

/var/folders/50/37d92zzj16z4b8zf67ydlch40000gn/T/ipykernel_31033/3536800266.py:7: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  output_summary = chain.run(docs)


In [12]:
output_summary

'John, a former graphic designer, struggles with unemployment, isolation, and a loss of self-worth after being laid off from his job at an advertising agency. As weeks turn into months, he becomes increasingly despondent, his savings dwindle, and he begins to retreat from his friends. One evening, after breaking down in tears, John decides he needs to make a change. The next morning, he wakes up early and starts writing, something he had always loved but never pursued. Although he knows his journey will be long, he feels a renewed sense of purpose and hope for the first time in a long time.'

## Summarizing Large Documents Using Map Reduce

In [13]:
from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [14]:
# provide the path of  pdf file/files.
pdfreader = PdfReader('/Users/mariyajose/Documents/Github_projects/text_summarisation/Code of Life.pdf')
from typing_extensions import Concatenate
# read text from pdf
text = ''
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        text += content

In [15]:
llm = ChatOpenAI(temperature=0, model_name='gpt-4')

In [16]:
llm.get_num_tokens(text)

1400

In [17]:
## Splittting the text
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=20)
chunks = text_splitter.create_documents([text])

In [18]:
len(chunks)

1

In [19]:
chain = load_summarize_chain(
    llm,
    chain_type='map_reduce',
    verbose=False
)
summary = chain.run(chunks)

In [20]:
summary

'Indian self-taught programmer, Samir Patel, studied at MIT and worked on AI projects in Silicon Valley. Due to ethical concerns and high pressure, he resigned and returned to India. There, he created a machine learning model to help diagnose rare diseases in underfunded rural areas, earning international recognition and funding from global health organizations. His story emphasizes the need for ethical considerations in technology and its potential for significant global impact.'

## Map Reduce With Custom Prompts

In [21]:
chunks_prompt="""
Please summarize the below story:
story:`{text}'
Summary:
"""
map_prompt_template=PromptTemplate(input_variables=['text'],
                                    template=chunks_prompt)

In [22]:
final_combine_prompt='''
Provide a final summary of the entire story with these important points.
Add a Generic Motivational Title,
Start the precise summary with an introduction and provide the
summary in number points for the story.
story: `{text}`
'''
final_combine_prompt_template=PromptTemplate(input_variables=['text'],
                                             template=final_combine_prompt)

In [23]:
summary_chain = load_summarize_chain(
    llm=llm,
    chain_type='map_reduce',
    map_prompt=map_prompt_template,
    combine_prompt=final_combine_prompt_template,
    verbose=False
)
output = summary_chain.run(chunks)

In [24]:
output

'Title: "Turning Challenges into Opportunities: The Inspiring Journey of Samir Patel"\n\nIntroduction: The story revolves around Samir Patel, a self-taught programmer from India, who navigates through the challenges of the tech industry to make a significant impact in the world. His journey from a small town in India to the prestigious halls of MIT and then to Silicon Valley, before returning to his roots, is a testament to his resilience and dedication.\n\n1. Samir Patel, hailing from a small town in India, had a deep-rooted fascination for puzzles and technology. His self-taught programming skills paved the way for his future endeavors.\n\n2. His dream of studying in the United States was realized when he was accepted into a prestigious program at MIT, specializing in artificial intelligence (AI).\n\n3. After graduation, Samir was recruited by a top tech company in Silicon Valley, marking a significant milestone in his career.\n\n4. Despite his professional success, Samir faced inten

## RefineChain For Summarization

In [25]:
chain = load_summarize_chain(
    llm=llm,
    chain_type='refine',
    verbose=True
)
output_summary = chain.run(chunks)



> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"Code of LifeThe BeginningSamir Patel had always been fascinated by puzzles. As a child, while other kids spent their afternoons playing outside, Samir would sit with his father’s old computer, a relic from the late 90s, solving logic puzzles or tinkering with lines of code he barely understood. The machine was slow, and the software was outdated, but to Samir, it was a portal to a world where the impossible became possible with a few keystrokes.Growing up in a small town in India, Samir's access to modern technology was limited, but his curiosity knew no bounds. He taught himself basic programming through whatever resources he could find—books from the local library, articles from tech magazines, and eventually, online tutorials when his town finally got a reliable internet connection. By the time he reached high school, Samir was alrea

In [26]:
output_summary

'Samir Patel, a self-taught programmer from a small town in India, pursued his passion for technology and moved to the United States to study at MIT. After graduation, he was recruited by a top tech company in Silicon Valley, where he worked on cutting-edge AI projects. However, the intense pressure and ethical dilemmas he faced in the industry led him to resign. He returned to India and developed a machine learning model to assist in diagnosing rare diseases in underfunded rural areas. His work received international recognition and funding, reaffirming his belief in using technology responsibly and ethically to make a meaningful difference in the world.'